<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Aerial_Photograph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install libraries for mosaic**

In [12]:
!pip install rasterio
!pip install matplotlib


**Import libraries to perform spatial mosaic and image classification**

In [13]:
import ee
import folium
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import os
from IPython.display import Image
import geemap
import rasterio
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.plot import show
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Authenticate google earth engine and your intialize the cloud project**

In [18]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

**Code to run a spatial mosaic, images are imported from google drive**

In [ ]:
input_dir = '/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB'

#create out out directory (run once)
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/Data/RGB_2530BB/mosaic

#List all GeoTIFF files in the directory
tif_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')] print(tif_files)

#Open all GeoTIFF files
src_files_to_mosaic = [rasterio.open(tif) for tif in tif_files]

#Merge the GeoTIFF files using a resampling method
mosaic, out_trans = merge(src_files_to_mosaic, resampling=Resampling.nearest) print(out_trans)

#Output file path for the mosaic
output_path = '/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif'

#Update metadata for the mosaic using the first GeoTIFF file in the list
out_meta = src_files_to_mosaic[0].meta.copy() out_meta.update({"driver": "GTiff", "height": mosaic.shape[1], "width": mosaic.shape[2], "transform": out_trans, "crs": src_files_to_mosaic[0].crs})

#Write the mosaic to a new GeoTIFF file
with rasterio.open(output_path, "w", **out_meta) as dest: dest.write(mosaic)

print('Progress done')

**Load image from GEE asset**

In [19]:
# Define the image
image = ee.Image('/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif')

In [17]:
#Load the image from Google Drive
image = ee.Image('projects/thandeka-skosana/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif');

#Print the image to check if it's loaded correctly
print(image);

map = geemap.Map(center=[20, 0], zoom=3)

# Define the bands to use for visualization
bands = ee.List(["b1", "b2", "b3"])

# Define visualization parameters
vis_params = {
    'bands': ['b1', 'b2', 'b3'],
    'min': 28,
    'max': 220,
}

display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)



ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "projects/thandeka-skosana/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif"
      }
    }
  }
})


EEException: Image.load: Image asset 'projects/thandeka-skosana/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif' not found (does not exist or caller does not have access).

**Visualize RGB Scene**

In [8]:
#visualize the RGB scene
# Define the bands to use for visualization
bands = ee.List(["b1", "b2", "b3"])

# Define visualization parameters
vis_params = {
    'bands': ['b1', 'b2', 'b3'],
    'min': 28,
    'max': 220,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

EEException: Image.load: Image asset '/content/drive/My Drive/Colab Notebooks/Data/RGB_2530BB/mosaic/out_put.tif' not found (does not exist or caller does not have access).

**Compute vegetation indices**

In [ ]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('b2'),
    'RED': image.select('b1'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('b1'),
    'GREEN': image.select('b2'),
    'BLUE': image.select('b3'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('b3'),
    'Red': image.select('b1'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo(), 'image_bands')

{'type': 'Image', 'bands': [{'id': 'b1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [21366, 23367], 'crs': 'PROJCS["Projection: Transverse Mercator; Datum: WGS84; Ellipsoid: WGS84", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Transverse_Mercator"], \n  PARAMETER["central_meridian", 31.0], \n  PARAMETER["latitude_of_origin", 0.0], \n  PARAMETER["scale_factor", 1.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'crs_transform': [0.25, 0, -10228.609567726324, 0, -0.25, -2788059.9809737173]}, {'id': 'b2', 'data_type': {'type': 'PixelType'

**Import training data from GEE**

In [ ]:

# Load the training data (e.g., a FeatureCollection) from GEE
training_data = ee.FeatureCollection('projects/mapwaps-sabiecroc/assets/Training_setSc')

# Convert the training data to a GeoDataFrame
features = training_data.getInfo()['features']
training_subset = gpd.GeoDataFrame.from_features(features)

# Display the GeoDataFrame
print(training_subset.head())



                     geometry DIRECTION DIRECTIO_1      DateTime DateTime_1  \
0  POINT (30.83475 -25.53864)                      -2.209133e+12              
1  POINT (30.83556 -25.53828)                      -2.209133e+12              
2  POINT (30.94377 -25.06340)         E            -2.209133e+12              
3  POINT (30.93543 -25.06455)         E            -2.209133e+12              
4  POINT (30.97520 -24.95397)         E             1.691478e+12              

   Distance_1  Distance_t Done Done_1  FID_  ...    Val_id Val_id_1  \
0           0           0                 0  ...  Val_4440            
1           0           0                 0  ...  Val_4441            
2           0           5    Y            0  ...    Val_19            
3           0          30    Y            0  ...    Val_23            
4           0          15    Y            0  ...    Val_33            

   Validation          X          Y        Z  Z_1  coords_x1 coords_x2  \
0           0   0.000000

**Sample regions from the image at training data locations**

In [ ]:
#select bands
bands = ['b1', 'b2', 'b3', 'GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI' ]

# Sample regions from the image at training feature locations
training_subset = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

**Run the classification using random forest**

In [ ]:
"""Classification using random forest"""

#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_subset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

**Defining a colour palette for the classified map and viweing the map**

In [ ]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=9)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m

Map(center=[-25.22500266876454, 30.924995571083596], controls=(WidgetControl(options=['position', 'transparent…